In [ ]:
!pip install pandas scikit-learn matplotlib seaborn

In [2]:
!pip install transformers datasets

In [3]:
!pip install nlpaug

In [4]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot

Jupyter command `jupyter-nbextension` not found.


In [5]:
!pip install torch

In [50]:
import pandas as pd

# CSV dosyasını oku
df = pd.read_csv("niyet_verisi.csv", encoding="windows-1254", sep=",")

# Veriye genel bakış
print(df.head())
print(df["label"].value_counts())


   id                                   text     label
0   1              "Bu ürünün fiyatı nedir?"   "bilgi"
1   2                 "Bugün hava çok güzel"   "bilgi"
2   3   "Yarın hangi saatlerde açıyorsunuz?"   "bilgi"
3   4       "Ürünümü nasıl iade edebilirim?"   "bilgi"
4   5    "Bu restoranda hangi yemekler var?"   "bilgi"
label
"bilgi"      4025
"şikayet"    4025
Name: count, dtype: int64


In [51]:
# Temizleme: ?şikayet gibi etiketleri düzelt
df["label"] = df["label"].str.replace("?", "", regex=False).str.strip()

In [52]:
import re
import nltk
from nltk.corpus import stopwords

# NLTK stopwords listesini indir (ilk seferde çalıştırman yeterli)
# nltk.download("stopwords")
# nltk.download("punkt")

# Türkçe stopwords listesi
turkce_stopwords = set(stopwords.words("turkish"))

# Emoji temizleme regex'i
def emojileri_temizle(metin):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # yüz ifadeleri
        u"\U0001F300-\U0001F5FF"  # sembol ve piktogramlar
        u"\U0001F680-\U0001F6FF"  # ulaşım ve harita sembolleri
        u"\U0001F1E0-\U0001F1FF"  # bayraklar (ülke kodları)
        u"\U00002702-\U000027B0"  # diğer semboller
        u"\U000024C2-\U0001F251" 
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', metin)

def temizle(metin):
    metin = metin.lower()  # küçük harfe çevir
    metin = re.sub(r"<.*?>", "", metin)  # HTML etiketlerini kaldır
    metin = re.sub(r"http\S+|www\.\S+", "", metin)  # URL'leri kaldır
    metin = emojileri_temizle(metin)  # emojileri temizle
    metin = re.sub(r"[^\w\s]", "", metin)  # noktalama işaretlerini kaldır
    metin = re.sub(r"\s+", " ", metin)  # fazla boşlukları sadeleştir

    # Stopwords çıkarımı
    kelimeler = metin.strip().split()
    kelimeler = [kelime for kelime in kelimeler if kelime not in turkce_stopwords]
    return " ".join(kelimeler)

df["text"] = df["text"].apply(temizle)


In [53]:
from sklearn.model_selection import train_test_split

X = df["text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [55]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Parametreler
epochs = 300
batch_size = 64
classes = np.unique(y_train)

# Model
model = MultinomialNB()

# Verileri numpy dizilerine dönüştür
X_data = X_train_tfidf
y_data = y_train.to_numpy()

# Accuracy değerlerini sakla
epoch_accuracies = []

# Epoch döngüsü
for epoch in range(epochs):
    print(f"📦 Epoch {epoch + 1}/{epochs}")
    
    # Veriyi karıştır
    indices = np.arange(X_data.shape[0])
    np.random.shuffle(indices)
    X_data = X_data[indices]
    y_data = y_data[indices]
    
    # Mini batch'lerle eğit
    for i in range(0, X_data.shape[0], batch_size):
        X_batch = X_data[i:i + batch_size]
        y_batch = y_data[i:i + batch_size]
        
        if epoch == 0 and i == 0:
            model.partial_fit(X_batch, y_batch, classes=classes)
        else:
            model.partial_fit(X_batch, y_batch)

    # Epoch sonunda doğruluk hesapla
    y_pred_epoch = model.predict(X_data)
    epoch_acc = accuracy_score(y_data, y_pred_epoch)
    epoch_accuracies.append(epoch_acc)
    print(f"📈 Epoch {epoch + 1} Accuracy: {epoch_acc:.4f}")

# Test verisi ile tahmin
y_pred = model.predict(X_test_tfidf)

# Sınıflandırma raporu
print("\n🔍 Sınıflandırma Raporu:\n")
print(classification_report(y_test, y_pred))

# Test accuracy
final_test_accuracy = accuracy_score(y_test, y_pred)
print(f"\n📊 Ortalama Eğitim Doğruluğu: {final_test_accuracy:.4f}")

# Ort. eğitim doğruluğu
mean_train_accuracy = np.mean(epoch_accuracies)
print(f"✅ Final Test Accuracy: {mean_train_accuracy:.4f}")

📦 Epoch 1/300
📈 Epoch 1 Accuracy: 0.9736
📦 Epoch 2/300
📈 Epoch 2 Accuracy: 0.9798
📦 Epoch 3/300
📈 Epoch 3 Accuracy: 0.9832
📦 Epoch 4/300
📈 Epoch 4 Accuracy: 0.9849
📦 Epoch 5/300
📈 Epoch 5 Accuracy: 0.9865
📦 Epoch 6/300
📈 Epoch 6 Accuracy: 0.9870
📦 Epoch 7/300
📈 Epoch 7 Accuracy: 0.9879
📦 Epoch 8/300
📈 Epoch 8 Accuracy: 0.9882
📦 Epoch 9/300
📈 Epoch 9 Accuracy: 0.9888
📦 Epoch 10/300
📈 Epoch 10 Accuracy: 0.9894
📦 Epoch 11/300
📈 Epoch 11 Accuracy: 0.9898
📦 Epoch 12/300
📈 Epoch 12 Accuracy: 0.9898
📦 Epoch 13/300
📈 Epoch 13 Accuracy: 0.9898
📦 Epoch 14/300
📈 Epoch 14 Accuracy: 0.9901
📦 Epoch 15/300
📈 Epoch 15 Accuracy: 0.9901
📦 Epoch 16/300
📈 Epoch 16 Accuracy: 0.9902
📦 Epoch 17/300
📈 Epoch 17 Accuracy: 0.9902
📦 Epoch 18/300
📈 Epoch 18 Accuracy: 0.9904
📦 Epoch 19/300
📈 Epoch 19 Accuracy: 0.9905
📦 Epoch 20/300
📈 Epoch 20 Accuracy: 0.9905
📦 Epoch 21/300
📈 Epoch 21 Accuracy: 0.9907
📦 Epoch 22/300
📈 Epoch 22 Accuracy: 0.9908
📦 Epoch 23/300
📈 Epoch 23 Accuracy: 0.9908
📦 Epoch 24/300
📈 Epoch 24 Acc

In [56]:
def niyet_tahmini(cumle):
    cumle = temizle(cumle)
    vec = vectorizer.transform([cumle])
    tahmin = model.predict(vec)
    return tahmin[0]

print(niyet_tahmini("Hayırsız.")) # Beklenen: Şikayet
print(niyet_tahmini("Fatura geldi.")) # Beklenen: Bilgi
print(niyet_tahmini("İlgilenmiyolar resmen ya.")) # Beklenen: Şikayet
print(niyet_tahmini("Ne zaman bi işim düşse, hep kayıplara karışıyo bu adam, bulamiyomki hiç.")) # Beklenen: Şikayet
print(niyet_tahmini("Dün de söyledim sana, gene aynı hatayı yapıyon işte, dinlemiyonki beni.")) # Beklenen: Şikayet
print(niyet_tahmini("iki artı iki dörttür.")) # Beklenen: Bilgi
print(niyet_tahmini("Beş para etmezsin.")) # Beklenen: Şikayet

"şikayet"
"bilgi"
"şikayet"
"şikayet"
"şikayet"
"bilgi"
"şikayet"


In [57]:
import joblib

# Eğittiğin modelin ve vektörizer'ın isimlerini değiştirmen gerekebilir
joblib.dump(model, "chatbot_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

['tfidf_vectorizer.pkl']